# Linear Regression 

In [8]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np

In [ ]:
train_file=r'~/Dropbox/0.0 Data/loan_data_train.csv'
test_file=r'~/Dropbox/0.0 Data/loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)               

In [ ]:
ld_test.head()

In [ ]:
ld_train.head()

In [ ]:
# lets combine the data for data prep
ld_train['data']='train'
ld_test['data']='test'
ld_all=pd.concat([ld_train,ld_test],axis=0)

In [ ]:
ld_all.head()

In [ ]:
ld_all.info()

In [ ]:
ld_all.head()

In [ ]:
# Amount.Funded.By.Investors ,ID: drop this 

# Amount.Requested, Open.CREDIT.Lines , Revolving.CREDIT.Balance: 
#.      change this to numeric 

# Debt.To.Income.Ratio , Interest.Rate : remove % and then convert 
# to numeric

# State , Loan.Purpose , Loan.Length , Home.Ownership, Employment.Lenth 
#. : make dummies 

# FICO.Range : break it in two columns ( covert to numeric) and then 
# take average 

In [ ]:
ld_all.drop(['ID','Amount.Funded.By.Investors'],axis=1,inplace=True)

In [ ]:
for col in ['Interest.Rate','Debt.To.Income.Ratio']:
    ld_all[col]=ld_all[col].str.replace("%","")
    
    

In [ ]:
for col in ['Amount.Requested', 'Interest.Rate','Debt.To.Income.Ratio',
            'Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
    ld_all[col]=pd.to_numeric(ld_all[col],errors='coerce')
    

In [ ]:
k=ld_all['FICO.Range'].str.split("-",expand=True).astype(float)

ld_all['fico']=0.5*(k[0]+k[1])

del ld_all['FICO.Range']



In [ ]:
# Notice that to apply string function on pandas data frame columns you need to str attribute
cat_cols=ld_all.select_dtypes(['object']).columns

In [ ]:
cat_cols

In [ ]:
cat_cols=cat_cols[:-1]

In [ ]:
cat_cols

In [ ]:
ld_all['State'].value_counts()

In [ ]:
#in next section for logistic regression we will be using  pandas' get dummies function. 
# you can work with either of these . 
#ignoring categories with low frequencies however will result in fewer columns without 
# affecting model performance too much .

for col in cat_cols:
    
    freqs=ld_all[col].value_counts()
    
    k=freqs.index[freqs>20][:-1]
    
    for cat in k:
        name=col+'_'+cat
        ld_all[name]=(ld_all[col]==cat).astype(int)
        
    del ld_all[col]
    print(col)
    
    
    

In [ ]:
ld_all.shape

In [ ]:
ld_all.isnull().sum()

In [ ]:
# ld_all.loc[ld_all['Monthly.Income']>5000,'Monthly.Income']


In [ ]:
for col in ld_all.columns:
    if (col not in ['Interest.Rate','data'])& (ld_all[col].isnull().sum()>0):
        ld_all.loc[ld_all[col].isnull(),col]=ld_all.loc[ld_all['data']=='train',col].mean()

In [ ]:
# ld_all.isnull().sum()

In [ ]:
ld_train=ld_all[ld_all['data']=='train']
del ld_train['data']
ld_test=ld_all[ld_all['data']=='test']
ld_test.drop(['Interest.Rate','data'],axis=1,inplace=True)

In [ ]:
del ld_all

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
ld_train1,ld_train2=train_test_split(ld_train,test_size=0.2,random_state=2)

In [ ]:
# Notice that only train data is used for imputing missing values in both train and test 
x_train1=ld_train1.drop('Interest.Rate',axis=1)
y_train1=ld_train1['Interest.Rate']

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm=LinearRegression()

In [ ]:
lm.fit(x_train1,y_train1)

In [ ]:
x_train1.shape

In [ ]:
# this is beta_0
lm.intercept_

In [ ]:
lm.coef_

In [ ]:
a=[1,2,3]
b=["bg","vj","ngfxfng"]

In [ ]:
list(zip(a,b))

In [ ]:
list(zip(x_train1.columns,lm.coef_))

In [ ]:
x_train2=ld_train2.drop('Interest.Rate',axis=1)

In [ ]:
predicted_ir=lm.predict(x_train2)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(ld_train2['Interest.Rate'],predicted_ir)

We know the tentative performance now, lets build the model on entire training to make prediction on test/production

In [ ]:
x_train=ld_train.drop('Interest.Rate',axis=1)
y_train=ld_train['Interest.Rate']

In [ ]:
lm.fit(x_train,y_train)

In [ ]:
test_pred=lm.predict(ld_test)

We can write these to a csv file for submission like this :

In [ ]:
# pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

In [ ]:
import os
os.getcwd()

# Ridge  Regression

In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [ ]:
lambdas=np.linspace(1,100,100)

In [ ]:
params={'alpha':lambdas}

In [ ]:
model=Ridge(fit_intercept=True)

In [ ]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring='neg_mean_absolute_error')

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
import sklearn

In [ ]:
sklearn.metrics.SCORERS.keys()

In [ ]:
# statsmodels 

In [ ]:
grid_search.best_estimator_

In [ ]:
# grid_search.cv_results_['std_test_score']

 if you want you can now fit a ridge regression model with obtained value of alpha , although there is no need, grid search automatically fits the best estimator on the entire data, you can directly use this to make predictions on test_data. But if you want to look at coefficients , its much more convenient to fit the model with direct function

Using the report function given below you can see the cv performance of top few models as well, that will the tentative performance

In [9]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
test_pred=grid_search.predict(ld_test)

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

## For looking at coefficients

In [ ]:
grid_search.best_estimator_

In [ ]:
ridge_model=grid_search.best_estimator_

In [ ]:
ridge_model.fit(x_train,y_train)

In [ ]:
list(zip(x_train1.columns,ridge_model.coef_))

## Lasso Regression

In [ ]:
lambdas=np.linspace(1,10,100)
model=Lasso(fit_intercept=True)
params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,
                         scoring='neg_mean_absolute_error')

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

you can see that, the best value of alpha comes at the edge of the range that we tried , we should expand the trial range on that side and run this again

In [ ]:
lambdas=np.linspace(.001,2,100)
params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
lasso_model=grid_search.best_estimator_

In [ ]:
lasso_model.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,lasso_model.coef_))

In [ ]:
(lasso_model.coef_==0).sum()

In [ ]:
x_train.shape

# Logistic Regression

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_file=r'~/Dropbox/0.0 Data/rg_train.csv'
test_file=r'~/Dropbox/0.0 Data/rg_test.csv'
bd_train=pd.read_csv(train_file)

bd_test=pd.read_csv(test_file)

In [6]:
bd_train['Revenue.Grid'].value_counts(normalize=True)

2    0.893772
1    0.106228
Name: Revenue.Grid, dtype: float64

In [10]:
# bd_test['Revenue.Grid']=np.nan
bd_train['data']='train'
bd_test['data']='test'
# bd_test=bd_test[bd_train.columns]
bd_all=pd.concat([bd_train,bd_test],axis=0)

In [11]:
bd_all['Revenue.Grid'].value_counts()

2.0    7261
1.0     863
Name: Revenue.Grid, dtype: int64

In [12]:
list(zip(bd_all.columns,bd_all.dtypes,bd_all.nunique()))

[('Average.A.C.Balance', dtype('float64'), 2223),
 ('Average.Credit.Card.Transaction', dtype('float64'), 1411),
 ('Balance.Transfer', dtype('float64'), 2183),
 ('Home.Loan', dtype('float64'), 884),
 ('Investment.Tax.Saving.Bond', dtype('float64'), 832),
 ('Investment.in.Commudity', dtype('float64'), 3558),
 ('Investment.in.Derivative', dtype('float64'), 3796),
 ('Investment.in.Equity', dtype('float64'), 3250),
 ('Investment.in.Mutual.Fund', dtype('float64'), 2470),
 ('Life.Insurance', dtype('float64'), 3111),
 ('Medical.Insurance', dtype('float64'), 1589),
 ('Online.Purchase.Amount', dtype('float64'), 1319),
 ('Personal.Loan', dtype('float64'), 1760),
 ('Portfolio.Balance', dtype('float64'), 8317),
 ('REF_NO', dtype('int64'), 10155),
 ('Revenue.Grid', dtype('float64'), 2),
 ('TVarea', dtype('O'), 14),
 ('Term.Deposit', dtype('float64'), 1419),
 ('age_band', dtype('O'), 13),
 ('children', dtype('O'), 5),
 ('data', dtype('O'), 2),
 ('family_income', dtype('O'), 13),
 ('gender', dtype('O'

In [16]:
# bd_all['post_code'].describe()
bd_all['children'].value_counts(dropna=False)

Zero    6208
1       1848
2       1607
3        473
4+        19
Name: children, dtype: int64

In [20]:
# bd_all['family_income'].value_counts()

In [21]:
# REF_NO,post_code , post_area  : drop 
# children : Zero : 0 , 4+ : 4 and then convert to numeric
# age_band : dummies 
# status , occupation , occupation_partner , home_status,family_income : dummies
# self_employed, ` : dummies
# TVArea , Region , gender : dummies
# Revenue Grid : 1,2 : 1,0

# try another iteration of this where you convert 
# age_band and family_income to numeric columns

In [22]:
bd_all.drop(['REF_NO','post_code','post_area'],axis=1,inplace=True)

In [23]:
bd_all['children']=np.where(bd_all['children']=='Zero',0,
                            bd_all['children'])
bd_all['children']=np.where(bd_all['children'].str[:1]=='4',4,
                            bd_all['children'])
bd_all['children']=pd.to_numeric(bd_all['children'],errors='coerce')

In [24]:
bd_all['Revenue.Grid']=(bd_all['Revenue.Grid']==1).astype(int)

In [25]:
cat_vars=bd_all.select_dtypes(['object']).columns


In [26]:
cat_vars

Index(['TVarea', 'age_band', 'data', 'family_income', 'gender', 'home_status',
       'occupation', 'occupation_partner', 'region', 'self_employed',
       'self_employed_partner', 'status'],
      dtype='object')

In [27]:
cat_vars=list(cat_vars)
cat_vars.remove('data')

In [33]:
# pd.get_dummies(bd_all['region'],prefix='region',drop_first=True)

In [34]:
for col in cat_vars:
    
    dummy=pd.get_dummies(bd_all[col],drop_first=True,prefix=col)
    
    bd_all=pd.concat([bd_all,dummy],axis=1)
    
    del bd_all[col]
    print(col)

del dummy

TVarea
age_band
family_income
gender
home_status
occupation
occupation_partner
region
self_employed
self_employed_partner
status


In [35]:
bd_all.shape

(10155, 96)

In [36]:
bd_all.isnull().sum().sum()

0

In [37]:
bd_train=bd_all[bd_all['data']=='train']
del bd_train['data']
bd_test=bd_all[bd_all['data']=='test']
bd_test.drop(['Revenue.Grid','data'],axis=1,inplace=True)

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.01,1000,10)}

In [40]:
2*2*10

40

In [41]:
model=LogisticRegression(fit_intercept=True)

In [42]:
from sklearn.model_selection import GridSearchCV

In [43]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,
                         scoring="roc_auc",n_jobs=-1,verbose=20)

In [44]:
x_train=bd_train.drop('Revenue.Grid',axis=1)
y_train=bd_train['Revenue.Grid']


In [45]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:   35.5s
[Paralle

[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 271 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 273 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 281 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  1.2min
[Paralle

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'class_weight': ['balanced', None], 'penalty': ['l1', 'l2'], 'C': array([1.0000e-02, 1.1112e+02, 2.2223e+02, 3.3334e+02, 4.4445e+02,
       5.5556e+02, 6.6667e+02, 7.7778e+02, 8.8889e+02, 1.0000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=20)

In [46]:
grid_search.best_estimator_

LogisticRegression(C=0.01, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [47]:
logr=grid_search.best_estimator_

In [48]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.95471 (std: 0.01283)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l1'}

Model with rank: 2
Mean validation score: 0.95416 (std: 0.01296)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.94884 (std: 0.01547)
Parameters: {'C': 555.56, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 4
Mean validation score: 0.94876 (std: 0.01544)
Parameters: {'C': 1000.0, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 5
Mean validation score: 0.94862 (std: 0.01521)
Parameters: {'C': 666.67, 'class_weight': 'balanced', 'penalty': 'l2'}



In [49]:
logr.fit(x_train,y_train)

LogisticRegression(C=0.01, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [52]:
x_train.shape

(8124, 94)

In [50]:
(logr.coef_[0]==0).sum()

81

In [53]:
list(zip(x_train.columns,logr.coef_[0]))

In [54]:
cutoffs=np.linspace(0.01,0.99,99)

cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [55]:
logr.predict_proba(x_train)

array([[0.99110361, 0.00889639],
       [0.85537641, 0.14462359],
       [0.95743783, 0.04256217],
       ...,
       [0.40540815, 0.59459185],
       [0.94944107, 0.05055893],
       [0.98027256, 0.01972744]])

In [56]:
logr.classes_

array([0, 1])

In [57]:
train_score=logr.predict_proba(x_train)[:,1]
real=y_train
 # In  order to find the probability of which column is for outcome 1 and which for outcome 0


In [58]:
KS_all=[]


for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
    
      
    KS=(TP/P)-(FP/N)
       
    KS_all.append(KS)
    
        


In [ ]:
# list(zip(cutoffs,KS_all))

In [59]:
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff

0.5

In [ ]:
# logr.intercept_

In [ ]:
# list(zip(x_train.columns,logr.coef_[0]))

if you simply had to submit probability scores , you could do this 

In [ ]:
logr.predict_proba(bd_test)

In [ ]:
test_score=logr.predict_proba(bd_test)[:,1]
test_score

In [ ]:
# pd.DataFrame(test_score).to_csv("mysubmission.csv",index=False)

if you had to submit hardclasses , you can apply the cutoff obtained above and then submit

In [ ]:
test_classes=(test_score>mycutoff).astype(int)

In [ ]:
# pd.DataFrame(test_classes).to_csv("mysubmission.csv",index=False)